In [1]:
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import joblib
% matplotlib inline

In [2]:
train_data = pd.read_csv('../data/unzipped/train.csv')

In [3]:
train_data.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


In [4]:
train_data.image[0]

'd10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c7679f17c333c959b19'

In [4]:
train_data.description.fillna(' ', inplace=True)

In [5]:
min(train_data['description'].apply(lambda x: len(str(x))))

1

In [6]:
max(train_data['description'].apply(lambda x: len(str(x))))

3212

In [7]:
test_data = pd.read_csv('../data/unzipped/test.csv')

In [8]:
test_data.description.fillna(' ', inplace=True)

In [10]:
max(test_data['description'].apply(lambda x: len(str(x))))

3206

In [9]:
concat_data = pd.concat([train_data, test_data], axis=0)

In [12]:
concat_data.head()

,activation_date,category_name,city,deal_probability,description,image,image_top_1,item_id,item_seq_number,param_1,param_2,param_3,parent_category_name,price,region,title,user_id,user_type
0,2017-03-28,Товары для детей и игрушки,Екатеринбург,0.12789,"Кокон для сна малыша,пользовались меньше месяц...",d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,b912c3c6a6ad,2,Постельные принадлежности,NaN,NaN,Личные вещи,400.0,Свердловская область,Кокоби(кокон для сна),e00f8ff2eaf9,Private
1,2017-03-26,Мебель и интерьер,Самара,0.00000,"Стойка для одежды, под вешалки. С бутика.",79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,2dac0150717d,19,Другое,NaN,NaN,Для дома и дачи,3000.0,Самарская область,Стойка для Одежды,39aeb48f0017,Private
2,2017-03-20,Аудио и видео,Ростов-на-Дону,0.43177,"В хорошем состоянии, домашний кинотеатр с blu ...",b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,ba83aefab5dc,9,"Видео, DVD и Blu-ray плееры",NaN,NaN,Бытовая электроника,4000.0,Ростовская область,Philips bluray,91e2f88dd6e3,Private
3,2017-03-25,Товары для детей и игрушки,Набережные Челны,0.80323,Продам кресло от0-25кг,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,02996f1dd2ea,286,Автомобильные кресла,NaN,NaN,Личные вещи,2200.0,Татарстан,Автокресло,bf5cccea572d,Company
4,2017-03-16,Автомобили,Волгоград,0.20797,Все вопросы по телефону.,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,7c90be56d2ab,3,С пробегом,ВАЗ (LADA),2110,Транспорт,40000.0,Волгоградская область,"ВАЗ 2110, 2003",ef50846afc0b,Private


In [10]:
char_list = []

In [11]:
for description in concat_data.description.iteritems():
    for char in description[1]:
        char_list.append(char)

In [12]:
len(char_list)

365517052

In [16]:
len(concat_data)

2011862

# ラベルエンコードする

In [14]:
le = LabelEncoder()
le.fit(char_list)

LabelEncoder()

In [19]:
label_encoded_list = []
for i, data in concat_data.iterrows():
    encoded = le.transform(list(data.description))
    label_encoded_list.append({'item_id': data.item_id, 'encoded':encoded})

In [20]:
joblib.dump(label_encoded_list, 'label_encoded.pkl', compress=True)

['label_encoded.pkl']

joblib.dump

In [21]:
label_encoded_list[0]

{'encoded': array([273, 309, 305, 309, 308,   2, 299, 306, 326,   2, 312, 308, 295,
          2, 307, 295, 306, 322, 319, 295,  14, 310, 309, 306, 323, 302,
        309, 297, 295, 306, 303, 312, 323,   2, 307, 300, 308, 323, 319,
        300,   2, 307, 300, 312, 326, 317, 295,  16, 317, 297, 300, 313,
          2, 312, 300, 311, 322, 304]), 'item_id': 'b912c3c6a6ad'}

In [18]:
from sklearn.preprocessing import LabelBinarizer

In [19]:
lb = LabelBinarizer()
lb.fit(char_list)

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [21]:
len(lb.classes_)

1749

In [55]:
import math
for t in concat_data.param_2.iteritems():
    print(math.isnan(t[1]))
    break

True


In [22]:
import pickle

In [23]:
with open('../data/pickle/char_onehot.pkl', mode='wb') as f:
    pickle.dump(lb, f, protocol=4)

In [24]:
for c in lb.classes_:
    print(c)

	


 
!
"
#
$
%
&
'
(
)
*
+
,
-
.
/
0
1
2
3
4
5
6
7
8
9
:
;
<
=
>
?
@
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
U
V
W
X
Y
Z
[
\
]
^
_
`
a
b
c
d
e
f
g
h
i
j
k
l
m
n
o
p
q
r
s
t
u
v
w
x
y
z
{
|
}
~
 
¡
¢
£
¤
¥
¦
§
¨
©
«
¬
­
®
¯
°
±
²
³
´
µ
¶
·
¸
¹
º
»
¼
½
¾
¿
Á
Ã
Ä
È
É
Ë
Ð
Ñ
Ó
Ö
×
Ø
Ú
Ü
ß
à
á
ä
å
ç
è
é
ê
ë
ì
í
ï
ñ
ò
ó
ô
ö
÷
ø
ù
û
ü
ý
ā
ą
Č
ē
Ę
ę
Ğ
ğ
ĥ
ī
İ
ı
ĸ
ĺ
Ł
ł
Ń
ń
ņ
ň
Ō
ō
Ő
Œ
ř
Ś
ś
Š
š
ų
Ÿ
Ź
ź
Ż
ż
ž
ƒ
Ɣ
Ƽ
Ǿ
Ȣ
Ʌ
ɑ
ə
ɟ
ʺ
˄
ˉ
˚
˜
˝
ˣ
˭
ˮ
́
̆
̈
̊
̋
̨
̶
͇
ͦ
;
·
Β
Γ
Δ
Λ
Ο
Π
Τ
Φ
Ω
έ
ή
α
β
γ
δ
θ
ι
κ
λ
μ
ο
ς
σ
τ
υ
φ
ω
Ϭ
Ё
Є
Ѕ
І
А
Б
В
Г
Д
Е
Ж
З
И
Й
К
Л
М
Н
О
П
Р
С
Т
У
Ф
Х
Ц
Ч
Ш
Щ
Ъ
Ы
Ь
Э
Ю
Я
а
б
в
г
д
е
ж
з
и
й
к
л
м
н
о
п
р
с
т
у
ф
х
ц
ч
ш
щ
ъ
ы
ь
э
ю
я
ѐ
ё
є
ѕ
і
ў
ѣ
ґ
қ
ҡ
ү
Ӏ
Ә
ә
ө
՞
ա
դ
ի
ծ
կ
ղ
ն
ո
ր
ւ
փ
ք
־
ا
ب
د
ر
م
ن
ه
٩
٭
۝
۞
۩
۫
۶
৪
੩
੫
૨
૭
ஐ
ஜ
౦
౩
๑
ლ
ღ
ᐁ
ᐃ
ᗒ
ᗕ
ᗣ
ᴓ
ᵌ
ᵒ
ồ
ἐ
 
 
 
 
 
​
‌
‍
‎
‏
‐
‑
‒
–
—
―
‗
‘
’
‚
‛
“
”
„
†
‡
•
‣
․
…
 
 
‪
‬
‭
 
‰
′
″
‹
›
※
‼
‾
‿
⁃
⁄
⁉
⁠
⁰
⁶
⁻
₁
₂
₊
₩
€
₱
₺
₽
⃣
℃
℅
№
℗
™
Ω
ℹ
⅛
⅜
Ⅰ
Ⅱ
Ⅲ
Ⅳ
Ⅴ
←
↑
→
↓
↔
↕
↗
↘
↙
↪
⇇
⇉
⇒
⇓
⇔
⇘
⇙
⇛
⇨
∅
∆
∎
∑
−
∙
√
∞
∨
∫
∮
≈
≡
≤


In [26]:
train_data.head

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


In [5]:
print(len(train_data), len(test_data))

1503424 508438


In [5]:
concat_data.head()

,activation_date,category_name,city,deal_probability,description,image,image_top_1,item_id,item_seq_number,param_1,param_2,param_3,parent_category_name,price,region,title,user_id,user_type
0,2017-03-28,Товары для детей и игрушки,Екатеринбург,0.12789,"Кокон для сна малыша,пользовались меньше месяц...",d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,b912c3c6a6ad,2,Постельные принадлежности,NaN,NaN,Личные вещи,400.0,Свердловская область,Кокоби(кокон для сна),e00f8ff2eaf9,Private
1,2017-03-26,Мебель и интерьер,Самара,0.00000,"Стойка для одежды, под вешалки. С бутика.",79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,2dac0150717d,19,Другое,NaN,NaN,Для дома и дачи,3000.0,Самарская область,Стойка для Одежды,39aeb48f0017,Private
2,2017-03-20,Аудио и видео,Ростов-на-Дону,0.43177,"В хорошем состоянии, домашний кинотеатр с blu ...",b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,ba83aefab5dc,9,"Видео, DVD и Blu-ray плееры",NaN,NaN,Бытовая электроника,4000.0,Ростовская область,Philips bluray,91e2f88dd6e3,Private
3,2017-03-25,Товары для детей и игрушки,Набережные Челны,0.80323,Продам кресло от0-25кг,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,02996f1dd2ea,286,Автомобильные кресла,NaN,NaN,Личные вещи,2200.0,Татарстан,Автокресло,bf5cccea572d,Company
4,2017-03-16,Автомобили,Волгоград,0.20797,Все вопросы по телефону.,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,7c90be56d2ab,3,С пробегом,ВАЗ (LADA),2110,Транспорт,40000.0,Волгоградская область,"ВАЗ 2110, 2003",ef50846afc0b,Private


In [10]:
item_id_dict = {}
for i, item in concat_data.item_id.iteritems():
    item_id_dict[item] = i

In [11]:
import pickle

with open('../data/pickle/item_id_dict.pkl', mode='wb') as f:
    pickle.dump(item_id_dict, f)

In [31]:
import pandas as pd

a = pd.Series([1,3,4,5])
b = pd.Series([24,5,56,6])

In [36]:
dataframe_test = pd.DataFrame()

In [39]:
dataframe_test['a'] = a
dataframe_test['b'] = b

In [40]:
dataframe_test

,a,b
0,1,24
1,3,5
2,4,56
3,5,6


In [22]:
import torch

In [24]:
torch.LongTensor([1,3,45,5]).shape

torch.Size([4])

In [4]:
import torch

data = torch.randn(5,2,4)

In [5]:
data.shape

torch.Size([5, 2, 4])

In [9]:
data = data.unsqueeze(1)
data.shape

torch.Size([5, 1, 2, 4])